# Connections Lab

### Introduction

In this lesson, we'll practice using airflow to connect to our AWS services.  Let's get started.

### Connecting to RDS

Now for this task, let's create a new dag that performs two tasks:

1. Create a task called `five_venues` that queries and logs the first five venues
2. Create a task called `three_categories` that queries and log the first three categories

To accomplish this, we can connect to an RDS instance already set up.  Here is some information you'll need to connect to the database.

In [1]:
host = "foursquare-flask-api.cbdkozm37vkd.us-east-1.rds.amazonaws.com"
database = 'postgres'
password = 'password1'

Ok, now let's create the dag and the two tasks, and use a PostgresHook to produce the requested information in the task logs.  If things are working properly, we can begin by seeing that the tasks have successfully performed.

<img src="./perform_queries.png" width="80%">

And then we can check the logs.

> So first we can see the values from `get_venues` returned.

<img src="./get_venues.png" width="100%">

> And then in the `get_categories` task, we can see that the values of Pizza, Italian, and Bar were returned.

<img src="./get_categories.png" width="90%">

Great, so we were successfully able to connect to our instances.

### Connecting to Redshift

Now let's try to connect to airflow to a redshift cluster.  And from there, we can even attempt to copy over some data from S3 over to redshift.

Let's get started.

1. Create the redshift cluster

To begin, create the redshift cluster by going [here](https://console.aws.amazon.com/redshift/home).  And then setup the cluster with the security group giving public access, and attaching an IAM role that gives read only access to S3.

2. Connect to the cluster

Then, before connecting to the cluster in airflow, let's make sure that we set everything up properly by connecting to the cluster with the `pyscopg2` library.  

Fill in the proper, information and create the connection.

In [4]:
import psycopg2

endpoint = "redshift-2.cdpgnoufdsdf.us-east-1.redshift.amazonaws.com"

conn = psycopg2.connect(
    host=endpoint,
    database="dev",
    port = "5439",
    user="awsuser",
    password="Password1")

In [5]:
cursor = conn.cursor()

3. Create an initial table

Then we'll need to create our tables in redshift.  For this lesson, we'll just practice copying over the zipcodes table, so let's create that table.

In [7]:
create_zipcodes_query = """CREATE TABLE "zipcodes" (
    "id" integer NOT NULL DEFAULT nextval('zipcodes_id_seq'),
    "code" INTEGER,
    "city_id" INTEGER
);"""

In [8]:
cursor.execute(create_zipcodes_query)

In [10]:
conn.commit()

4. Connect to redshift in airflow

Ok, now it's time to connect to redshift in airflow.  To do so, we'll need to: 

* Create a connection in airflow
* Create a task that copies data from airflow
    * To create the task, we can use the `connection.run` function, and the query below.
    > Fill in the s3 bucket name, and the `aws_iam_role` value.

In [ ]:
qry = """COPY zipcodes (id, code, city_id) from 's3://jigsaw-sample-data/zipcodes.csv'
credentials 'aws_iam_role=arn:aws:iam::095598444804:role/myRedshiftRole'
delimiter ','
IGNOREHEADER 1
region 'us-east-1';
"""

cursor.execute(qry)

* Then after creating a task that runs the copy command, create another task that selects from redshift's zipcodes table to confirm the zipcodes were copied over.

So in summary, we should now have a dag with three tasks: the first queries our rds, the second copies over data from s3 to redshift, and the third queries the data in redshift.

So if we look at a tree view of our DAG, we should see the following:

> <img src="./airflow_dag.png" width="60%">

And if we ultimately view the log of the `select_zipcodes` task, we should see our zipcodes loaded in there.

> <img src="./returned_zipcodes.png" width="90%">

### Summary

In this lesson, we practiced working with and setting up connections so that we could connect to our RDS database.  In future lessons, we'll see how we can use the connections to perform our steps of loading data RDS to S3 and S3 to redshift.